In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import xml 


In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import re


In [44]:

driver =webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.researchgate.net/login')

def fb_login():
    user,psw=  '#','#' 
    driver.get ('https://www.researchgate.net/login')
    driver.find_element_by_id('input-login').send_keys(user)
    driver.find_element_by_id('input-password').send_keys(psw)
    driver.find_element_by_class_name('action-submit').click()





====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/workplace/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache


In [70]:
# dep 3: https://www.researchgate.net/institution/The_University_of_Queensland/department/School_of_Social_Science/publications
# dep 4: https://www.researchgate.net/institution/University-of-Brighton/department/School-of-Applied-Social-Science/publications
# dep 5: https://www.researchgate.net/institution/University-of-Roehampton/department/Department-of-Social-Science/publications
# dep 6: https://www.researchgate.net/institution/Liverpool_John_Moores_University/department/School_of_Humanities_and_Social_Science/publications
# https://www.researchgate.net/institution/UNSW-Sydney/department/School-of-Humanities-and-Social-Science/publications
# https://www.researchgate.net/institution/York_University/department/Department_of_Social_Science/publications
# https://www.researchgate.net/institution/University-College-London/department/Department-of-Social-Science/publications
#https://www.researchgate.net/institution/University-of-Aberdeen/department/School-of-Social-Science/publications
#https://www.researchgate.net/institution/Universiti_Putra_Malaysia/department/Department_of_Social_and_Development_Science/publications



#not scraped yet
# https://www.researchgate.net/institution/York_University/department/Department_of_Social_Science/publications
#https://www.researchgate.net/institution/The-University-of-Hong-Kong/department/Department-of-Social-Work-and-Social-Administration2/publications
#https://www.researchgate.net/institution/The-University-of-York/department/Department-of-Social-Policy-and-Social-Work/publications
#https://www.researchgate.net/institution/University_of_Sussex/department/Department_of_Social_Work_and_Social_Care/publications
# 

In [45]:
fb_login()

In [46]:
driver.get('https://www.researchgate.net/institution/Deakin-University/department/School-of-Humanities-and-Social-Sciences/publications')

time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 1 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
i = 1

while True:
    # scroll one screen height each time
    driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
    i += 1
    time.sleep(scroll_pause_time)
    # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
    scroll_height = driver.execute_script("return document.body.scrollHeight;")  
    # Break the loop when the height we need to scroll to is larger than the total scroll height
    if (screen_height) * i > scroll_height:
        break 

stages = driver.find_elements_by_class_name('nova-legacy-c-card__body--spacing-inherit')


In [47]:
stages = list(set(stages))
len(stages)

425

In [48]:
cols=['title','authers','abstract','type','date_pub','journal','date_added','figuers','full_text','citation','interest','reads']
df=pd.DataFrame(columns=cols)
df

,title,authers,abstract,type,date_pub,journal,date_added,figuers,full_text,citation,interest,reads


In [49]:
df.head()

,title,authers,abstract,type,date_pub,journal,date_added,figuers,full_text,citation,interest,reads


In [50]:
df.shape

(0, 12)

In [51]:
t_class= 'nova-legacy-v-publication-item__title'
auth_class='nova-legacy-v-publication-item__person-list'
cat_class='nova-legacy-v-publication-item__badge'
f_class='nova-legacy-c-image-strip__image-magnifier'
txt_class='nova-legacy-c-button__label'
cit_class="nova-legacy-e-link--theme-bare"
reads_class="nova-legacy-e-link--theme-bare"



In [52]:
for s in stages:
    title=s.find_element_by_class_name(t_class)
    title=title.text
    print(title)

    #---

    auth=s.find_element_by_class_name(auth_class)
    auth=auth.text.split('\n')
    print(auth)


    #---

    cat=s.find_element_by_css_selector('.nova-legacy-e-badge--color-green.nova-legacy-e-badge--theme-solid.nova-legacy-e-badge--luminosity-high')
    cat=cat.text
    print(cat)

    #---
    
    txt=s.find_element_by_css_selector('.nova-legacy-o-pack--gutter-m>.nova-legacy-o-pack__item')
    txt=txt.text.replace('View','')
    print(txt)

    #---
    metrics=s.find_element_by_css_selector(".nova-legacy-v-publication-item__metrics")
    metrics=re.findall("\d+", metrics.text)

    if (len(metrics)==0):
        reads=0
        citations=0
    elif (len(metrics)==1):
        reads=metrics[0]
        citations=0
    else:
        reads=metrics[0]
        citations=metrics[1]
    
    print(f'reads : {reads}')
    print(f'citations : {citations}')


 
    #---
    try:
        figures=s.find_element_by_class_name(f_class)
        figures=1
    except:
        figures= 0
    
    print('figures:'+str(figures))

    #---
    try:
        date_added=s.find_element_by_css_selector('.nova-legacy-v-activity-item__time')
        date=date_added.text
        print(date)
    except:
        date= None
    #---
    date_pub=s.find_element_by_css_selector('.nova-legacy-v-publication-item__meta-data')
    date_pub=date_pub.text
    date_pub=re.sub('(\d+(\.\d+)?)', r' \1 ', date_pub).strip()
    year = re.findall("\d +", date_pub)
    journal= re.findall("\s\w+(.*)", date_pub)
    try:
        journal=journal[0]
    except:
        journal=None
    try:
        year=year[0]
    except:
        year=None

    #---

    # link class: nova-legacy-e-link--theme-bare
    #abstract class: nova-legacy-e-expandable-text__container
    link=s.find_element_by_class_name('nova-legacy-v-publication-item__title')
    link=link.find_element_by_class_name('nova-legacy-e-link--theme-bare')
    link=link.get_attribute('href')
    link
    
    # to get the current window handler for future to return
    curwin = driver.current_window_handle
    # open link in new tab keyboard shortcut
    driver.execute_script('''window.open("about:blank", "_blank");''')
    driver.switch_to.window(driver.window_handles[1])  # assuming new tab is at index 1
    # edit 1


    driver.get(link)
    #.nova-legacy-e-expandable-text__container
    #.nova-legacy-e-text--color-grey-900
    time.sleep(3)  
    try:
        abstract=driver.find_element_by_css_selector('.nova-legacy-e-expandable-text__container')
        abstract=abstract.text
    except:
        abstract=None 
    
    try:
        interest=driver.find_element_by_css_selector('.nova-legacy-e-text--color-grey-900')
        interest=re.findall("\d+\.\d+", interest.text)
        interest=interest[0]
    except:
        interest=None 
   
    print(interest)
    driver.close()  # closes new tab
    driver.switch_to.window(curwin)
    
    row_dict = {'title':title,'authers':auth,'abstract':abstract,'type':cat,'date_pub':year,'journal':journal,'date_added':date,'figuers':figures,'full_text':txt,'citation':citations,'interest':interest,'reads':reads} 
    df = df.append(row_dict, ignore_index = True)
    


Mapping technology-harm relations: From ambient harms to zemiosis
['Mark A. Wood']
Article

Request full-text
reads : 21
citations : 0
figures:0
Jul 17
0.9
Enhancing relationships between criminology and cybersecurity
['Benoit Dupont', 'Chad Whelan']
Article

Download
reads : 171
citations : 1
figures:0
Mar 27
3.9
Dancing for diversity: The mixed-race identity of Indonesian-Australian cultural performers
['Monika Winarnita']
Article

Request full-text
reads : 2
citations : 0
figures:0
May 7
None
Out of Time: Modernity, Historicity, and Temporality in Ernst Jünger’s War Journals
['Marilyn Stendera']
Chapter

Download
reads : 5
citations : 0
figures:0
0.3
The Molecular Turn, Predictive Technologies and Diagnostic Fragmentation: An Interview with David Armstrong
['Alan Petersen', 'Kiran Pienaar']
Article

Request full-text
reads : 2
citations : 0
figures:0
0.1
The changing role of the FIRB and the politics of foreign investment in Australia
['David Ross Hundt']
Article

Download
reads : 5

1.8
Los Jaivas: Toward A Decolonial Attitude in Chilean Psychedelic Rock
['Israel Holas', 'Sergio Holas']
Article

Download
reads : 0
citations : 0
figures:0
None
Hacia una estética infrarreal en la poesía temprana de Roberto Bolaño
['Israel Holas']
Article

Download
reads : 1
citations : 0
figures:0
None
‘Murderous Coppers’: Police, Industrial Disputes and the 1929 Rothbury Shootings
['Richard Evans']
Article

Request full-text
reads : 8
citations : 4
figures:0
Feb 17
2.0
There's a Woman in the House, A 1950's Journey
['Deborah Jane Jordan', 'L. Jordan', 'Sarah Paddle']
Article

Request full-text
reads : 1
citations : 0
figures:0
None
“I Just Don’t Wanna Deal With the Headache of People Fighting Over the Internet”: A Study of Sponsored Female Climbers’ Digital Labor
['Katariina Rahikainen', 'Kim Toffoletti']
Article

Request full-text
reads : 9
citations : 0
figures:0
Oct 5
0.1
Understanding the structure and composition of co-offending networks in Australia
['David Bright', 'Chad Whe

0.5
A Confrontation on the Cocos (Keeling) Islands: Interviewing, Local Language, and Rapport in Anthropological Fieldwork
['Nicholas Herriman', 'Monika Winarnita']
Chapter

Request full-text
reads : 10
citations : 0
figures:0
0.1
Book review: Science Fiction and Digital Technologies in Argentine and Brazilian Culture
['Israel Holas']
Article

Download
reads : 2
citations : 0
figures:0
None
Rethinking how Technologies Harm
['Mark A. Wood']
Article

Request full-text
reads : 49
citations : 1
figures:0
1.1
MENTAL HEALTH PROMOTION SERVICES FOR REFUGEES AND PEOPLE SEEKING ASYLUM IN AUSTRALIA
['Dickson Sebit']
Thesis

Download
reads : 269
citations : 0
figures:1
0.9
Ontological Quality Control in Large-scale, Applied Ontology Matching
['Catherine Legg', 'Samuel Sarjant']
Poster

Download
reads : 1
citations : 0
figures:0
None
Disintegrating the Linear: Time in Simon Finn's Instability
['Marilyn Stendera']
Article

Download
reads : 0
citations : 0
figures:0
Dec 2
None
Ontological Quality Con

['Alannah Buller', 'Sarah Epstein', 'Norah Hosken']
Article

Request full-text
reads : 23
citations : 0
figures:0
Apr 15
0.8
Mapping the interstices: intertextuality, language, and authorial voice in Zapatista poetics
['Eugenia Demuro', 'Israel Holas']
Article

Request full-text
reads : 13
citations : 1
figures:0
0.6
Fair and Equal Access to Justice: Institutional and Strategic Reform of Nepali Judiciary (Nepali)
['Bidhya Chapagain']
Article

Request full-text
reads : 27
citations : 0
figures:0
Jul 9
1.0
Building a humanitarian sector career: understanding the education vs experience tension
['Matthew Clarke', 'Sophie Perreard', 'Phil Connors']
Article

Request full-text
reads : 54
citations : 5
figures:0
2.8
Fresh groundwater lens dynamics of a small bedrock island in the tropics, Northern Australia
['Eddie Banks', 'Vincent Post', 'Karina Meredith[...]', 'Wen Eddie']
Article

Request full-text
reads : 22
citations : 0
figures:0
Jan 10
4.2
Farewell sweet ladybird: a manifesto and three

['Catherine Legg']
Presentation

Download
reads : 94
citations : 0
figures:0
Mar 2
3.6
The uncertain commodity of ‘security’: Are private security companies ‘value for money’ for domestic violence services?
['Diarmaid Harkin']
Article

Request full-text
reads : 3
citations : 0
figures:0
Oct 17
0.2
Notes on ultra-realism: A response to Raymen and Kuldova
['Briony Elise Anderson', 'Imogen Richards', 'Mark A. Wood']
Preprint

Download
reads : 115
citations : 0
figures:0
3.4
Problematising LGBTIQ drug use, governing sexuality and gender: A critical analysis of LGBTIQ health policy in Australia
['Kiran Pienaar', 'Dean A Murphy', 'Kane Race', 'Toby Lea']
Article

Download
reads : 131
citations : 24
figures:0
14.6
Grasping Mathematical Reality
['Catherine Legg']
Article

Download
reads : 0
citations : 0
figures:0
None
Towards a philosophy of academic publishing
['Michael Adrian Peters', 'Petar Jandric', 'Ruth Irwin[...]', 'Leon Benade']
Chapter

Download
reads : 121
citations : 0
figures:0
8.

1.4
Elected and Non-elected Representative Claim-Makers in Indonesia
['Michael Hatherell']
Chapter

Request full-text
reads : 9
citations : 0
figures:0
Jan 1
0.2
'Uninhibited Play' The political and pragmatic dimensions of intoxication within queer cultures
['Kane Race', 'Kiran Pienaar', 'Dean A Murphy', 'Toby Lea']
Preprint

Download
reads : 119
citations : 0
figures:0
2.8
A blessing in disguise: Beyond cross-cultural online supervision
['Citra Amelia', 'Alistair Welsh', 'Monika Winarnita', 'Ramón López Castellano']
Article

Request full-text
reads : 11
citations : 0
figures:0
Nov 30
0.3
Everyday peace: rethinking typologies of social practice and local agency
['Anthony Ware', 'Vicki Ware']
Article

Request full-text
reads : 2
citations : 0
figures:0
Nov 7
0.1
Enhancing relationships between criminology and cybersecurity
['Benoit Dupont', 'Chad Whelan']
Article

Download
reads : 171
citations : 1
figures:0
May 10
3.9
Injecting as a sexual practice: Cultural formations of ‘slamsex’
['K

1.1
A properly pragmatist pragmatics: Peircean reflections on the distinction between semantics and pragmatics
['Catherine Legg']
Article

Request full-text
reads : 9
citations : 0
figures:0
Oct 8
0.4
Conclusion
['George Duke']
Chapter

Request full-text
reads : 1
citations : 0
figures:0
Dec 13
None
Indonesian women journalists and precarious work
['Samiaji Bintang', 'Monika Winarnita', 'Ignatius Haryanto[...]', 'Albertus Magnus Prestianta']
Chapter

Request full-text
reads : 2
citations : 0
figures:0
0.1
CRG 52/14-15: Filicide in Australia, 2000-2012 A National Report
['Thea Brown', 'Samantha Lyneham', 'Willow Bryant[...]', 'Paula Fernandez Arias']
Technical Report

Download
reads : 3
citations : 0
figures:1
Dec 8
0.1
‘The footage is decisive’: Applying the thinking of Marshall McLuhan to CCTV and police misconduct
['Richard Evans']
Article

Request full-text
reads : 32
citations : 8
figures:0
4.2
Pleural Epithelioid Hemangioendothelioma: A Case Report
['Malik H Khan', 'Terrance Barne

['Mike Presdee', 'Reece Walters']
Article

Request full-text
reads : 2
citations : 21
figures:0
Jan 5
10.6
The Cambridge Companion to Natural Law Jurisprudence
['George Duke', 'R.P. George']
Book

Request full-text
reads : 9
citations : 9
figures:0
4.6
Alternative National Narratives in Indonesia
['Michael Hatherell', 'Alistair Welsh']
Chapter

Request full-text
reads : 9
citations : 0
figures:0
0.2
CRG 52/14-15: Filicide in Australia, 2000-2012 A National Report
['Thea Brown', 'Samantha Lyneham', 'Willow Bryant[...]', 'Paula Fernandez Arias']
Technical Report

Download
reads : 3
citations : 0
figures:0
0.1
Enhancing relationships between criminology and cybersecurity
['Benoit Dupont', 'Chad Whelan']
Article

Download
reads : 171
citations : 1
figures:0
3.9
Discursive Habits: Peirce and Cognitive Semiotics
['Catherine Legg']
Presentation

Download
reads : 94
citations : 0
figures:0
3.6
The Use of Digital Platforms for Adults' and Adolescents' Physical Activity During the COVID-19 Pande

0.7
Using Social Psychology to Constructively Involve Colleagues in Internal Evaluation
['Alison Rogers', 'Leanne Kelly', 'Alicia McCoy']
Article

Download
reads : 54
citations : 0
figures:0
2.5
A Confrontation on the Cocos (Keeling) Islands: Interviewing, Local Language, and Rapport in Anthropological Fieldwork
['Nicholas Herriman', 'Monika Winarnita']
Chapter

Request full-text
reads : 10
citations : 0
figures:0
Feb 18
0.1
Book review : one alliance, two lenses : US-Korea relations in a new era
['David Ross Hundt']
Article

Download
reads : 41
citations : 0
figures:0
Jun 3
0.2
Protecting the rights of LGBTIQ people around the world: Beyond marriage equality and the decriminalisation of homosexuality
['Paula Gerber', 'Senthorun Raj', 'Cai Wilkinson', 'Anthony J. Langlois']
Article

Request full-text
reads : 51
citations : 0
figures:0
0.9
CRG 52/14-15: Filicide in Australia, 2000-2012 A National Report
['Thea Brown', 'Samantha Lyneham', 'Willow Bryant[...]', 'Paula Fernandez Arias']
Te

0.1


| col | Description | Type |
| --- | --- | --- |
| title | article's title | string 
| auther| auther name | string 
| abstract | abstract full-text  | string
| category| article, literature review, conference paper..etc | string
| date_published | date the article was puplished | date
| date_added | date the article was uploaded to researchgate | date
| figuersCount | number of images in preview | int
| full_text? | avilability of full text ( using download or Request full-text as keywords) | string
| citation | number of times that paper was cited | int
| reads | number of views (target variable)| int



In [43]:
df.sample(10)

,title,authers,abstract,type,date_pub,journal,date_added,figuers,full_text,citation,interest,reads
58,Remember Afghanistan?,"[Andrew Hoskins, William Merrin]",None,Article,1,Digital War,Nov 21,0,\nRequest full-text,0,0,15
52,(Not) a good place to stay! – East European mi...,"[Rebecca Kay, Paulina Trevena]","In this article, we ask how migrants establish...",Article,1,Journal of Ethnic and Migration Studies,Feb 20,0,\nDownload,0,1,23
7,The COVID-19 pandemic: the essential role of t...,"[Claire Bynner, Maureen McBride, Sarah Weakley]",This research note highlights the need to enga...,Article,1,Voluntary Sector Review,Jul 19,0,\nRequest full-text,0,None,0
29,Book Review: Addictive Consumption: Capitalism...,[Gerda Reith],None,Article,1,Journal of Consumer Culture,May 28,0,\nRequest full-text,0,0,14
31,‘New’ Migrations Transforming the City: East E...,"[Rebecca Kay, Paulina Trevena]",None,Chapter,None,,None,0,\nRequest full-text,0,0,5
13,Editors’ note,"[Andrew Hoskins, Barry Richards, Philip Seib]",None,Article,9,"Media, War and Conflict",Feb 8,0,\nRequest full-text,0,0,2
75,Inhibition of myocardial cathepsin-L release d...,"[Weihong He, Charlotte McCarroll, Katrin Nathe...",Aims Identifying novel mediators of lethal myo...,Article,1,Cardiovascular Research,Jun 19,1,\nDownload,0,2,54
76,EXTENDING ECONOMIC CASES FOR HOUSING POLICIES:...,"[Duncan Maclennan, Jinqiao Long]",None,Technical Report,None,,None,1,\nDownload,3,1,10
55,Housing and productivity: all or nothing at all?,"[Duncan Maclennan, Jinqiao Long, Chris Leishman]",None,Technical Report,None,,None,1,\nDownload,0,3,82
23,Raising productivity and housing the economy,"[Duncan Maclennan, Julie Tian Miao, Linda Chri...",None,Chapter,None,,None,0,\nRequest full-text,2,1,4


In [34]:
df.head()

,title,authers,abstract,type,date_pub,journal,date_added,figuers,full_text,citation,interest,reads
0,PWE-103 Positive nutritional outcomes in patie...,"[Rebecca Whitefield, R Iriarte, H Bridgestock[...",Introduction HPN use in those with malignant b...,Conference Paper,7,Gut,Sep 23,0,\nRequest full-text,0,None,1
1,The beef with vegans: Managing stigma in Brita...,[Hannah Howard],As one of the fastest growing lifestyle choice...,Article,None,,None,0,\nDownload,0,2,145
2,Gender-based Violence in Sport: Prevalence and...,"[Melanie Lang, Lut Mergaert, Tine Vertommen, C...",Little is known about the prevalence of gender...,Article,1,European Journal for Sport and Society,Nov 10,0,\nDownload,0,2,47
3,Depression in adolescent obesity: Cultural mod...,"[J Wardle, S Williamson, Fiona Johnson, C Edwa...",Community studies relating depression to obesi...,Article,6,International Journal of Obesity,None,0,\nDownload,94,48,135
4,Caffeine addiction and its effects.,[Seyi Joseph],None,Article,1,Nursing Times,None,0,\nRequest full-text,2,1,27


In [53]:
df.shape

(425, 12)

In [54]:
df['title'].duplicated().sum()

225

In [55]:
df.drop_duplicates(subset=['title'],inplace=True)

In [56]:
df.shape

(200, 12)

In [57]:
df.to_csv('articles20.csv', index=False)

In [67]:
i=0
df1=pd.read_csv('articles.csv')

for i in range(2,22):
    df1=df1.append((pd.read_csv(f'articles{i}.csv')))
    i+=1



In [68]:
df1.shape

(1899, 12)

In [69]:
df1.duplicated().sum()

0

In [70]:
df1.to_csv('data2.csv', index=False)

In [66]:
df1=pd.read_csv('data.csv')



In [67]:
df1.sample(10)

,title,authers,abstract,type,date_pub,journal,date_added,figuers,full_text,citation,interest,reads
1077,Developing socially inclusive practices in Chi...,['Gaston Bacquet'],Overview for upcoming doctoral research to be ...,Presentation,NaN,NaN,NaN,0,\nDownload,0,0.0,6
564,"Longitudinal trends in PrEP familiarity, attit...","['Ian W Holloway', 'Evan A. Krueger', 'Ilan H ...","This study explored familiarity with, attitude...",Article,0.0,PLoS ONE,Jan 5,1,\nDownload,2,4.0,54
760,Human Trafficking: A Spotlight on Sex Trafficking,"['Dominique Roe-Sepowitz', 'Kristine E. Hickle']",NaN,Chapter,NaN,NaN,"Dec 25, 2020",0,\nRequest full-text,0,0.0,2
1104,Improving Research Methods Teaching and Learni...,"['Matt Ryan', 'Clare Saunders', 'Emily Rainsfo...",Student dissatisfaction with undergraduate res...,Article,3.0,Politics,NaN,0,\nRequest full-text,28,14.0,72
1019,Development of the Top Tips Habit-Based Weight...,"['Nathalie Kliemann', 'Helen Croker', 'Fiona J...",Background: The Ten Top Tips (10TT) is an inte...,Article,9.0,JMIR mhealth and uhealth,NaN,1,\nDownload,7,7.0,179
1054,The effects of pegylated recombinant human meg...,"['Michael W Schuster', 'Roy Beveridge', 'Debra...",To assess the safety and efficacy of pegylated...,Article,2.0,Experimental Hematology,NaN,0,\nRequest full-text,43,21.0,16
810,Abuse and Cruelty in Religious Bureaucracy: Th...,['Michael Salter'],NaN,Chapter,NaN,NaN,NaN,0,\nRequest full-text,0,0.0,7
596,Gender and Political Apology: When the Patriar...,['Emma Dolan'],NaN,Book,NaN,NaN,Jul 18,0,\nRequest full-text,2,1.0,12
553,Lifelong exposure to air pollution and cogniti...,"['Ai Milojevic', 'Peter Dutey-Magni', 'Lorrain...",Background. Evidence about the impact of air p...,Article,1.0,Environmental Research Letters,Feb 23,0,\nRequest full-text,1,1.0,27
1166,Numerical investigation on the thermal reliabi...,"['Tao Xi', 'Xingang Yu', 'Xifu Song[...]', 'Pi...",The reliability of printed circuit board level...,Article,2.0,International Journal of Materials and Struct...,NaN,0,\nRequest full-text,9,4.0,25


In [68]:
df1.shape

(1223, 12)

In [69]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1223 entries, 0 to 1222
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   title       1223 non-null   object 
 1   authers     1223 non-null   object 
 2   abstract    808 non-null    object 
 3   type        1223 non-null   object 
 4   date_pub    759 non-null    float64
 5   journal     694 non-null    object 
 6   date_added  757 non-null    object 
 7   figuers     1223 non-null   int64  
 8   full_text   1223 non-null   object 
 9   citation    1223 non-null   int64  
 10  interest    1037 non-null   float64
 11  reads       1223 non-null   int64  
dtypes: float64(2), int64(3), object(7)
memory usage: 114.8+ KB
